In [1]:
# TODO: Remove after development
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

import tests

In [3]:
tests.check_tf(tf)

Using TensorFlow 1.4.0


# Practice

### Basic tensor operations
* Create tensors from Python lists
* Create tensors from numpy arrays
* Get Tensor name, rank, shape
* Get tensor by name
* Create constant tensors
* Stack tensors
* ...

In [4]:
def create_tensor_from_list(list_):
    tensor = tf.convert_to_tensor(list_)
    return tensor

tests.test_create_tensor_from_list(create_tensor_from_list)

✓ Tests passed


In [5]:
def create_tensor_from_np_array(numpy_array, name):
    tensor = tf.convert_to_tensor(numpy_array, name=name)
    return tensor

tests.test_create_tensor_from_np_array(create_tensor_from_np_array)

✓ Tests passed


In [6]:
def get_tensor_name(tensor):
    name = tensor.name
    return name

tests.test_get_tensor_name(get_tensor_name)

✓ Tests passed


In [7]:
def get_tensor_shape(tensor):
    shape = tensor.shape
    return shape

tests.test_get_tensor_shape(get_tensor_shape)

✓ Tests passed


In [8]:
def get_tensor_rank(tensor):
    rank_tensor = tf.rank(tensor)
    rank_value = tf.Session().run(rank_tensor)
    return rank_value

tests.test_get_tensor_rank(get_tensor_rank)

✓ Tests passed


In [9]:
def create_constant_tensor(value, m, n):
    tensor = tf.constant(value, shape=[m, n])
    return tensor

tests.test_create_constant_tensor(create_constant_tensor)

✓ Tests passed


### Using scopes
Why even use scopes?
* Tensorboard
* Each scope its own ops
* Nesting possible

In [10]:
def create_variable_in_scope(name, np_array, scope_name):
    with tf.name_scope(scope_name):
        tensor = create_tensor_from_np_array(np_array, name)
        return tensor
        
tests.test_create_variable_in_scope(create_variable_in_scope)

✓ Tests passed


In [11]:
def create_variable_in_nested_scope(name, np_array, scope_name_outer, scope_name_inner):
    with tf.name_scope(scope_name_outer):
        with tf.name_scope(scope_name_inner):
            tensor = create_tensor_from_np_array(np_array, name)
            return tensor
        
tests.test_create_variable_in_nested_scope(create_variable_in_nested_scope)

✓ Tests passed


### Using multiple graphs

In [13]:
g = tf.Graph()
g

In [14]:
hex(id(g))

'0x11a743a58'

In [15]:
default_graph = tf.get_default_graph()
default_graph

In [16]:
hex(id(default_graph))

'0x11a708630'

In [17]:
def get_default_graph():
    graph = tf.get_default_graph()
    return graph

tests.test_get_default_graph(get_default_graph)

✓ Tests passed


In [18]:
def create_new_graph():
    graph = tf.Graph()
    return graph

tests.test_create_new_graph(create_new_graph)

✓ Tests passed


In [19]:
def get_graph_seed(graph):
    seed = graph.seed
    return seed

tests.test_get_graph_seed(get_graph_seed)

✓ Tests passed


In [20]:
def set_graph_seed(graph, seed):
    with graph.as_default():
        tf.set_random_seed(seed)
    return graph

tests.test_set_graph_seed(set_graph_seed)

✓ Tests passed


### Variables

In [36]:
# get_variable vs. Variable
# https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow

# collections
# https://www.tensorflow.org/programmers_guide/variables

# trainables
# an Optimizer can manipulate
# tf.trainable_variables()

# Variable shape
# TensorShape
# TensorShape == [row, col, ...]

### Placeholders

In [ ]:
graph = tf.Graph()

with tf.Session(graph=graph) as sess:
    x = tf.placeholder(tf.float32, [1, 3])
    y = tf.sign(x)
    z = sess.run(y, feed_dict={x: [[3, 3, -1]]})

### Randomness
Operations that rely on a random seed actually derive it from two seeds: the graph-level and operation-level seeds. This sets the graph-level seed.
- Seed
- ...

### Data Types

### Sessions

### Math operations

In [21]:
def add(x, y):
    return x + y

tests.test_add(add)

✓ Tests passed


In [22]:
def add_rank0_tensors(x, y):
    tensor_x = tf.constant(x)
    tensor_y = tf.constant(y)
    tensor_z = tensor_x + tensor_y
    return tensor_z

tests.test_add_rank0_tensors(add_rank0_tensors)

✓ Tests passed


In [23]:
def add_rank1_tensors(x, y):
    tensor_x = tf.constant(x)
    tensor_y = tf.constant(y)
    tensor_z = tensor_x + tensor_y
    return tensor_z

tests.test_add_rank1_tensors(add_rank1_tensors)

✓ Tests passed


### Reduce operations

- reduce_sum
- reduce_mean

### Other stuff

In [ ]:
tf.tensordot(tf.constant([1,2,3]), tf.constant([1,2,3]), axis=[[1], [0]])

### Perceptron

In [202]:
labels = np.array([[1,1,1,0],
                   [1,1,1,0],
                   [1,1,1,0],
                   [1,1,1,0]], dtype=np.uint8)

predictions = np.array([[1,0,0,0],
                        [1,1,0,0],
                        [1,1,1,0],
                        [0,1,1,1]], dtype=np.uint8)

n_batches = len(labels)

In [204]:
graph = tf.Graph()
with graph.as_default():
    # Placeholders to take in batches onf data
    tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
    tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

    # Define the metric and update operations
    tf_metric, tf_metric_update = tf.metrics.accuracy(tf_label,
                                                      tf_prediction,
                                                      name="my_metric")

    # Isolate the variables stored behind the scenes by the metric operation
    running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")

    # Define initializer to initialize/reset running variables
    running_vars_initializer = tf.variables_initializer(var_list=running_vars)


with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())

    # initialize/reset the running variables
    session.run(running_vars_initializer)

    for i in range(n_batches):
        # Update the running variables on new batch of samples
        feed_dict={tf_label: labels[i], tf_prediction: predictions[i]}
        session.run(tf_metric_update, feed_dict=feed_dict)

    # Calculate the score
    score = session.run(tf_metric)
    print("[TF] SCORE: ", score)

[TF] SCORE:  0.6875


In [238]:
graph = tf.Graph()

with graph.as_default():
    # Placeholders to take in batches onf data
    tf_label = tf.convert_to_tensor(labels)
    tf_prediction = tf.convert_to_tensor(predictions)

    # Define the metric and update operations
    tf_metric, tf_metric_update = tf.metrics.accuracy(tf_label,
                                                      tf_prediction,
                                                      name="my_metric")

    # Isolate the variables stored behind the scenes by the metric operation
    running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")

    # Define initializer to initialize/reset running variables
    running_vars_initializer = tf.variables_initializer(var_list=running_vars)


with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())

    # initialize/reset the running variables
    session.run(running_vars_initializer)
    session.run(tf_metric_update)

    # Calculate the score
    score = session.run(tf_metric)
    print("[TF] SCORE: ", score)

[TF] SCORE:  0.6875


In [74]:
# Why use a bias?
# https://stats.stackexchange.com/questions/185911/why-are-bias-nodes-used-in-neural-networks
# a neural network does not have a bias node in a given layer, it will not be able to produce output in the next layer that differs from 00 (on the linear scale, or the value that corresponds to the transformation of 00 when passed through the activation function) when the feature values are 00.
# http://www.chioka.in/why-do-we-need-a-bias-neuron/
# Decision boundary translation

In [239]:
class PerceptronWithTF:
    def __init__(self, learning_rate, epochs):
        self.learning_rate = learning_rate
        self.epochs = epochs
    
    def fit(self, x, y):
        graph = tf.Graph()

        with graph.as_default():
            # w = tf.Variable(tf.random_normal([3, 1]), dtype=tf.float32)
            w = tf.Variable(tf.zeros([3, 1]), dtype=tf.float32)
            # x_ = np.hstack([x, np.ones([len(x), 1], dtype=np.float32)])
            x_ = np.c_[x, np.ones(len(x), dtype=np.float32)]
            y_hat = tf.sign(tf.matmul(x_, w))
            error = y - y_hat
            mse = tf.reduce_mean(tf.square(error))
            y_tensor = tf.convert_to_tensor(y)
            print(y_tensor.shape)
            print(y_hat.shape)
            print('=' * 10)
            accuracy, acc_op = tf.metrics.accuracy(y_tensor, y_hat)
            delta = tf.matmul(x_, error, transpose_a=True)
            train = tf.assign(w, tf.add(w, self.learning_rate * delta))

        with tf.Session(graph=graph) as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            for epoch in range(self.epochs):
                error_, w_, acc_ = sess.run([mse, train, acc_op])
                print('Epoch: {} MSE: {}, w: {}, acc: {}'.format(epoch + 1,
                                                                 error_,
                                                                 w_.reshape(1, len(w_)),
                                                                 0.1
                                                                ))

In [240]:
x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=np.float32)
y_and = np.array([[-1], [-1], [-1], [1]])
y_or = np.array([[-1], [1], [1], [1]])

In [241]:
learning_rate = 0.01
epochs = 10

In [242]:
perceptron = PerceptronWithTF(learning_rate, epochs)
perceptron.fit(x, y_and)

(4, 1)
(4, 1)


FailedPreconditionError: Attempting to use uninitialized value accuracy/total
	 [[Node: accuracy/AssignAdd = AssignAdd[T=DT_FLOAT, _class=["loc:@accuracy/total"], use_locking=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](accuracy/total, accuracy/Sum)]]

Caused by op 'accuracy/AssignAdd', defined at:
  File "/Users/amir.ziai/anaconda/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/amir.ziai/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/amir.ziai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/amir.ziai/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/amir.ziai/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/amir.ziai/anaconda/l

In [222]:
perceptron = PerceptronWithTF(learning_rate, epochs)
perceptron.fit(x, y_or)

(4, 1)
(4, 1)
Epoch: 1 MSE: 1.0, w: [[ 0.02  0.02  0.02]], acc: 0.1
Epoch: 2 MSE: 1.0, w: [[ 0.02  0.02  0.  ]], acc: 0.1
Epoch: 3 MSE: 0.25, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 4 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 5 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 6 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 7 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 8 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 9 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
Epoch: 10 MSE: 0.0, w: [[ 0.02  0.02 -0.01]], acc: 0.1
